Kitti dataset analysis
 

In [1]:
import os

In [2]:
datadir = 'training\label_2'

Classification Sataset
1. How many classes
2. How many images per class
3. Is there class imbalance

Object Detection Dataset
1. How many classes
2. How many bounding boxes per class
3. How many max bounding boxes per image and least bounding box in an image
4. Max, minimum bounding box size per class.


Below is propsed model to store and access kitti dataset

Kitti Dataset Annotations Format in Json

```
{
    images : {
        image_id : [
            {   
                name : criteriaName,
                Truncation : float, 
                Occlusion : int,
                Alpha : float, 
                BBox : list(int)[xmin, ymin, xmax, ymax], 
                3-D Dimention : List(float)[Height, width, length], 
                Location : [x, y, z], 
                Rotation : float
                annotation_id : id // newly vreated
            }
        ]
    }

    Criterias : {
        criteriaName : {
            <!-- id : int -->
            annotations : [annotation_id]
            images : [image_id]
        }
    }
}
```

Coco Like Format

```
{
    images : [
        {
            image_id : int
            file_name : string
            height : float
            width : float
        }
    ],

    annotations : [
        Truncation : float, 
        Occlusion : int,
        Alpha : float, 
        BBox : list(int)[xmin, ymin, xmax, ymax], 
        3-D Dimention : List(float)[Height, width, length], 
        Location : [x, y, z], 
        Rotation : float
        annotation_id : int
        image_id : int
        criteria_id : int
    ],

    Criterias : [
        {
            id : int
            Name : str
        }
    ]
}

```

In [ ]:
indexNameDict = {
    'Criteria': 0,
    'Truncation': 1,
    'Occlusion': 2,
    'Alpha': 3,
    'BBox': (4,8),
    '3d Dimensions': (8, 11),
    'Location': (11,14),
    'Rotation': 14
}

In [3]:
# index to name dictionary
import json

all_files = os.listdir(datadir)
imageDict = dict()
criteriaDict = dict()
for a in all_files:
    with open(os.path.join(datadir, a), 'r') as f:
        fileinput = f.read().split('\n')
        fileinput = [f.split(' ') for f in fileinput if len(f.split(' ')) == 15]
        imageDict[a[0:-4]] = list()
        for i, f in enumerate(fileinput):
            FileDict = dict()
            for k, v in indexNameDict.items():
                if type(v) is tuple:
                    FileDict[k] = f[v[0]: v[1]]
                else:
                    FileDict[k] = f[v]
                    if k == 'Criteria':
                        if f[v] not in criteriaDict.keys():
                            CriDict = dict()
                            CriDict['annotations'] = [a[0:-4] + str(i)]
                            CriDict['images'] = [a[0:-4]]
                            criteriaDict[f[v]] = CriDict
                        else:
                            criteriaDict[f[v]]['annotations'].append(a[0:-4] + str(i))
                            criteriaDict[f[v]]['images'].append(a[0:-4])
            FileDict['annotation_id'] = a[0:-4] + str(i)
            imageDict[a[0:-4]].append(FileDict)
            
DataDict = dict()
DataDict['images'] = imageDict
DataDict['criteria'] = criteriaDict
with open('Kitti.json', 'w') as f:
    json.dump(DataDict, f, indent = 4)


In [8]:
# dataDict
# range()
import json

with open('Kittidataimages.json', 'w') as fp:
    json.dump(imageDict, fp, indent=4)

In [9]:

import json

with open('Kittidatacriteria.json', 'w') as fp:
    json.dump(criteriaDict, fp, indent=4)

In [1]:
import KittiToCocoConvertor
KtDataset = KittiToCocoConvertor.KittiToCocoConvertor(path = 'training\label_2')
KtDataset.saveDataset(filePath = 'KittiInCoco.json')